# 1  Problem Statement

If they are more likely to be controlled the more frequent they visit and if that changes based on patient demographics like with insurance without insurance etc
We generally tell diabetics every 3 months want to see if that makes a difference with diabetes and last office visit and control vs. uncontrolled numbers. 


1.       PatientID repeats because they have more than one visit date and more than one date when their a1c was recorded.

2.       Summary = Document or office visit summary associated with the visit. I don’t know if that would be a HIPPA violation, but there is no patient information on the summary so I am guessing it should be good.

3.       VisitDate , summary, date of a1c  has repeated values because the date when a1c was recorded and the visit date are not necessarily the same and patient has different visit dates and different date of a1c, basically its two data sets one would be patientid, summary and visit and the other would be Patientid, date of a1c and a1c_value, Combining those two will create duplicates.

1. Are patients more likely to have a lower/controlled A1c if they have visited more frequently? 
2. NLP on summary: words like poor/worse 
3. Can we predict based off of frequency of visits or last visit, what the patients A1c is? 
- people who come in more frequently, more inclined to have lower levels 
- controlled randomised experiment 

# Imports

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV,Ridge, Lasso
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 500)
%matplotlib inline

In [127]:
df=pd.read_csv("../data/A1c_Data_Requesta1c.csv")

In [131]:
df_2=pd.read_csv("../data/diabetes.csv")

In [132]:
df_2.head()

PatientId VisitDate                        SUMMARY   OBSDATE Date of A1c
0         72   7/19/17  followup diabetes, med refill   7/21/17         8.1
1         72   7/19/17  followup diabetes, med refill   1/19/19         7.6
2         72   7/19/17  followup diabetes, med refill    2/8/18         8.3
3         72   7/19/17  followup diabetes, med refill  10/20/17         7.7
4         72   7/19/17  followup diabetes, med refill    6/8/19         7.1

In [128]:
df.head(25)

PatientId  Age           Birthdate           VisitDate  \
0          72   56  11/26/1963 5:00:00  7/19/2017 14:30:00   
1          72   56  11/26/1963 5:00:00  7/19/2017 14:30:00   
2          72   56  11/26/1963 5:00:00  7/19/2017 14:30:00   
3          72   56  11/26/1963 5:00:00  7/19/2017 14:30:00   
4          72   56  11/26/1963 5:00:00  7/19/2017 14:30:00   
5          72   56  11/26/1963 5:00:00  7/19/2017 14:30:00   
6          72   56  11/26/1963 5:00:00  7/19/2017 14:30:00   
7          72   56  11/26/1963 5:00:00  7/19/2017 14:30:00   
8          72   56  11/26/1963 5:00:00  7/19/2017 14:30:00   
9          72   56  11/26/1963 5:00:00  7/19/2017 14:30:00   
10         72   56  11/26/1963 5:00:00  7/19/2017 14:30:00   
11         72   56  11/26/1963 5:00:00  7/19/2017 14:30:00   
12         72   56  11/26/1963 5:00:00  7/28/2017 15:30:00   
13         72   56  11/26/1963 5:00:00  7/28/2017 15:30:00   
14         72   56  11/26/1963 5:00:00  7/28/2017 15:30:00   
15         72   56  11/26/1963 5:00:00  7/28/2017 15:30:00   
16         72   56  11/26/1963 5:00:00  7/28/2017 15:30:00   
17         72   56  11/26/1963 5:00:00  7/28/2017 15:30:00   
18         72   56  11/26/1963 5:00:00  7/28/2017 15:30:00   
19         72   56  11/26/1963 5:00:00  7/28/2017 15:30:00   
20         72   56  11/26/1963 5:00:00  7/28/2017 15:30:00   
21         72   56  11/26/1963 5:00:00  7/28/2017 15:30:00   
22         72   56  11/26/1963 5:00:00  7/28/2017 15:30:00   
23         72   56  11/26/1963 5:00:00  7/28/2017 15:30:00   
24         72   56  11/26/1963 5:00:00  11/8/2017 14:30:00   

                          SUMMARY              OBSDATE  Date of A1c  \
0   followup diabetes, med refill   7/21/2017 17:21:00          8.1   
1   followup diabetes, med refill   7/21/2017 17:21:00          8.1   
2   followup diabetes, med refill    1/19/2019 0:01:15          7.6   
3   followup diabetes, med refill    1/19/2019 0:01:15          7.6   
4   followup diabetes, med refill    2/8/2018 19:50:00          8.3   
5   followup diabetes, med refill    2/8/2018 19:50:00          8.3   
6   followup diabetes, med refill  10/20/2017 17:54:00          7.7   
7   followup diabetes, med refill  10/20/2017 17:54:00          7.7   
8   followup diabetes, med refill    6/8/2019 11:34:19          7.1   
9   followup diabetes, med refill    6/8/2019 11:34:19          7.1   
10  followup diabetes, med refill   12/9/2019 14:36:12          8.2   
11  followup diabetes, med refill   12/9/2019 14:36:12          8.2   
12      f/u lab results, DM2, HLD   12/9/2019 14:36:12          8.2   
13      f/u lab results, DM2, HLD   12/9/2019 14:36:12          8.2   
14      f/u lab results, DM2, HLD    6/8/2019 11:34:19          7.1   
15      f/u lab results, DM2, HLD    6/8/2019 11:34:19          7.1   
16      f/u lab results, DM2, HLD    2/8/2018 19:50:00          8.3   
17      f/u lab results, DM2, HLD    2/8/2018 19:50:00          8.3   
18      f/u lab results, DM2, HLD  10/20/2017 17:54:00          7.7   
19      f/u lab results, DM2, HLD  10/20/2017 17:54:00          7.7   
20      f/u lab results, DM2, HLD    1/19/2019 0:01:15          7.6   
21      f/u lab results, DM2, HLD    1/19/2019 0:01:15          7.6   
22      f/u lab results, DM2, HLD   7/21/2017 17:21:00          8.1   
23      f/u lab results, DM2, HLD   7/21/2017 17:21:00          8.1   
24  DM and dyslipidemia follow-up   7/21/2017 17:21:00          8.1   

             Insurance  
0   Dental Sliding Fee  
1    Sliding Fee Scale  
2   Dental Sliding Fee  
3    Sliding Fee Scale  
4   Dental Sliding Fee  
5    Sliding Fee Scale  
6   Dental Sliding Fee  
7    Sliding Fee Scale  
8   Dental Sliding Fee  
9    Sliding Fee Scale  
10  Dental Sliding Fee  
11   Sliding Fee Scale  
12  Dental Sliding Fee  
13   Sliding Fee Scale  
14  Dental Sliding Fee  
15   Sliding Fee Scale  
16  Dental Sliding Fee  
17   Sliding Fee Scale  
18  Dental Sliding Fee  
19   Sliding Fee Scale  
20  Dental Sliding 

# Cleaning and EDA

- Discovered 1,099 patients to look at
-Changed column names
-Changed object type to integer for a1c levels 
-Removed 1.7 a1c
-Changed .089 to 8.9 -- assume an error, 8.9 also the mean value 
-Change date object to datetime 
-drop index 

Moving forward
- group a1c into 8 and below= controlled
- 9 and above = uncontrolled
- add as another column and save into csv 
- You can ignore it and assumed that the visit date is the same as the day they get their A1c- inconsistent 
- group a few patients of different age groups and export to CVS to do EDA 


In [4]:
#Check number of patients by ID
df["PatientId"].value_counts()

115425    6072
971       5880
386       5247
111755    4422
201709    3410
          ... 
234272       2
241167       2
233853       1
234976       1
233208       1
Name: PatientId, Length: 1099, dtype: int64

In [5]:
#change column names 
df.rename(columns={"PatientId": "patient_id","Age": "age", "VisitDate": "visit_date", "SUMMARY":"summary","OBSDATE":"obs_date","Date of A1c":"date_of_a1c","Insurance":"insurance"},inplace=True)

In [35]:
df.tail(50)

index  patient_id  age           Birthdate           visit_date  \
272851  273435      243193   67   1/14/1953 5:00:00  12/31/2019 13:45:00   
272852  273436      243193   67   1/14/1953 5:00:00  12/31/2019 13:45:00   
272853  273437      243193   67   1/14/1953 5:00:00  12/31/2019 13:45:00   
272854  273438      243193   67   1/14/1953 5:00:00  12/31/2019 13:45:00   
272855  273439      243204   65   4/21/1954 5:00:00    1/13/2020 8:20:00   
272856  273440      243204   65   4/21/1954 5:00:00    1/13/2020 8:20:00   
272857  273441      243204   65   4/21/1954 5:00:00    1/13/2020 8:20:00   
272858  273442      243204   65   4/21/1954 5:00:00   1/24/2020 13:00:00   
272859  273443      243204   65   4/21/1954 5:00:00   1/24/2020 13:00:00   
272860  273444      243204   65   4/21/1954 5:00:00   1/24/2020 13:00:00   
272861  273445      243204   65   4/21/1954 5:00:00   1/31/2020 13:40:00   
272862  273446      243204   65   4/21/1954 5:00:00   1/31/2020 13:40:00   
272863  273447      243204   65   4/21/1954 5:00:00   1/31/2020 13:40:00   
272864  273448      243204   65   4/21/1954 5:00:00    2/5/2020 11:00:00   
272865  273449      243204   65   4/21/1954 5:00:00    2/5/2020 11:00:00   
272866  273450      243204   65   4/21/1954 5:00:00    2/5/2020 11:00:00   
272867  273451      243240   41   3/23/1978 5:00:00   12/23/2019 9:30:00   
272868  273452      243240   41   3/23/1978 5:00:00   12/23/2019 9:30:00   
272869  273453      243240   41   3/23/1978 5:00:00     1/3/2020 9:00:00   
272870  273454      243240   41   3/23/1978 5:00:00     1/3/2020 9:00:00   
272871  273455      243240   41   3/23/1978 5:00:00   1/30/2020 11:00:00   
272872  273456      243240   41   3/23/1978 5:00:00   1/30/2020 11:00:00   
272873  273457      243410   59    1/5/1961 5:00:00  11/27/2019 14:45:00   
272874  273458      243410   59    1/5/1961 5:00:00  11/27/2019 14:45:00   
272875  273459      243548   53   6/29/1966 4:00:00   11/6/2019 14:15:00   
272876  273460      243548   53   6/29/1966 4:00:00   11/6/2019 14:15:00   
272877  273461      243548   53   6/29/1966 4:00:00   11/6/2019 14:15:00   
272878  273462      243548   53   6/29/1966 4:00:00   12/2/2019 10:15:00   
272879  273463      243548   53   6/29/1966 4:00:00   12/2/2019 10:15:00   
272880  273464      243548   53   6/29/1966 4:00:00   12/2/2019 10:15:00   
272881  273465      243548   53   6/29/1966 4:00:00   12/24/2019 8:15:00   
272882  273466      243548   53   6/29/1966 4:00:00   12/24/2019 8:15:00   
272883  273467      243548   53   6/29/1966 4:00:00   12/24/2019 8:15:00   
272884  273468      243548   53   6/29/1966 4:00:00    1/9/2020 13:45:00   
272885  273469      243548   53   6/29/1966 4:00:00    1/9/2020 13:45:00   
272886  273470      243548   53   6/29/1966 4:00:00    1/9/2020 13:45:00   
272887  273471      243642   54    1/9/1966 5:00:00   1/13/2020 14:15:00   
272888  273472      243642   54    1/9/1966 5:00:00   1/13/2020 14:15:00   
272889  273473      243642   54    1/9/1966 5:00:00    1/28/2020 9:00:00   
272890  273474      243642   54    1/9/1966 5:00:00    1/28/2020 9:00:00   
272891  273475      243644   58  12/20/1961 5:00:00  11/20/2019 13:45:00   
272892  273476      243644   58  12/20/1961 5:00:00  11/20/2019 13:45:00   
272893  273477      243644   58  12/20/1961 5:00:00  11/27/2019 16:59:00   
272894  273478      243644   58  12/20/1961 5:00:00  11/27/2019 16:59:00   
272895  273479      243644   58  12/20/1961 5:00:00  12/10/2019 14:40:00   
272896  273480      243644   58  12/20/1961 5:00:00  12/10/2019 14:40:00   
272897  273481      243644   58  12/20/1961 5:00:00  12/11/2019 13:45:00   
272898  273482      243644   58  12/20/1961 5:00:00  12/11/2019 13:45:00   
272899  273483      243644   58  12/20/1961 5:00:00   1/10/2020 13:45:00   
272900  273484      243644   58  12/20/1961 5:00:00   1/10/2020 13:45:00   

                                                  summary  \
272851  f/u of renal lesion-need repeat MRI 6/20, proc...   
272852  f/

In [7]:
df.shape

(273485, 8)

In [8]:
df.dtypes

patient_id       int64
age              int64
Birthdate       object
visit_date      object
summary         object
obs_date        object
date_of_a1c    float64
insurance       object
dtype: object

In [9]:
df.isnull().sum()

patient_id       0
age              0
Birthdate        0
visit_date       0
summary        218
obs_date         0
date_of_a1c     32
insurance        0
dtype: int64

In [10]:
df.dropna(subset=["date_of_a1c"],inplace=True)

In [11]:
df.reset_index(inplace=True)

In [12]:
df.isnull().sum()

index            0
patient_id       0
age              0
Birthdate        0
visit_date       0
summary        218
obs_date         0
date_of_a1c      0
insurance        0
dtype: int64

In [13]:
#convert object to float 
def convert(a1c_obj):
    a1c_num=float(a1c_obj)
    return a1c_num

In [14]:
df["date_of_a1c"].map(convert)

0          8.1
1          8.1
2          7.6
3          7.6
4          8.3
          ... 
273448    13.4
273449    13.4
273450    12.2
273451    12.2
273452    13.4
Name: date_of_a1c, Length: 273453, dtype: float64

In [15]:
df["date_of_a1c"].sort_values(ascending=True)

204598     0.089
204659     0.089
204658     0.089
204651     0.089
204650     0.089
           ...  
268928    19.700
268929    19.700
268930    19.700
268931    19.700
268927    19.700
Name: date_of_a1c, Length: 273453, dtype: float64

In [16]:
df["date_of_a1c"].value_counts(ascending=True)

19.700        5
15.200       16
17.000       20
15.400       34
17.100       39
15.300       40
16.900       44
0.089        45
13.600       60
14.300      107
14.500      115
14.900      116
14.600      120
15.000      120
13.200      122
15.500      164
14.700      180
13.800      202
13.500      223
15.600      224
13.900      234
16.700      252
12.800      264
14.100      298
14.800      317
14.200      346
14.000      354
14.400      427
13.100      437
1.700       552
11.900      573
13.700      579
13.400      627
12.900      719
12.700      733
13.300      863
12.300     1010
12.600     1044
11.600     1097
13.000     1267
11.700     1413
12.000     1456
12.400     1526
10.900     1613
12.500     1615
11.300     1772
12.200     1847
11.500     1859
11.800     1893
12.100     1942
11.000     2001
11.400     2134
11.200     2235
10.600     2357
11.100     2528
10.800     2679
9.800      2734
10.400     2749
10.300     2961
10.000     3273
10.700     3307
9.000      3422
9.700   

In [17]:
df["date_of_a1c"].max()

19.7

In [18]:
df["date_of_a1c"].min()
#this minimum is too low, assume this was meant to be 8.9, which is the mean of the dataset 

0.08900000000000001

In [19]:
df["date_of_a1c"].describe()

count    273453.000000
mean          8.902426
std           1.760240
min           0.089000
25%           7.500000
50%           8.400000
75%           9.900000
max          19.700000
Name: date_of_a1c, dtype: float64

In [20]:
mask_1= df["date_of_a1c"]== 0.08900000000000001
#instead of deleting, change 

In [21]:
mask_1

0         False
1         False
2         False
3         False
4         False
          ...  
273448    False
273449    False
273450    False
273451    False
273452    False
Name: date_of_a1c, Length: 273453, dtype: bool

In [22]:
df["date_of_a1c"].where(df["date_of_a1c"]!= 0.08900000000000001, 8.9,inplace=True)

In [23]:
df.head()

index  patient_id  age           Birthdate          visit_date  \
0      0          72   56  11/26/1963 5:00:00  7/19/2017 14:30:00   
1      1          72   56  11/26/1963 5:00:00  7/19/2017 14:30:00   
2      2          72   56  11/26/1963 5:00:00  7/19/2017 14:30:00   
3      3          72   56  11/26/1963 5:00:00  7/19/2017 14:30:00   
4      4          72   56  11/26/1963 5:00:00  7/19/2017 14:30:00   

                         summary            obs_date  date_of_a1c  \
0  followup diabetes, med refill  7/21/2017 17:21:00          8.1   
1  followup diabetes, med refill  7/21/2017 17:21:00          8.1   
2  followup diabetes, med refill   1/19/2019 0:01:15          7.6   
3  followup diabetes, med refill   1/19/2019 0:01:15          7.6   
4  followup diabetes, med refill   2/8/2018 19:50:00          8.3   

            insurance  
0  Dental Sliding Fee  
1   Sliding Fee Scale  
2  Dental Sliding Fee  
3   Sliding Fee Scale  
4  Dental Sliding Fee

In [24]:
df["date_of_a1c"].value_counts(ascending=True)

19.7        5
15.2       16
17.0       20
15.4       34
17.1       39
15.3       40
16.9       44
13.6       60
14.3      107
14.5      115
14.9      116
15.0      120
14.6      120
13.2      122
15.5      164
14.7      180
13.8      202
13.5      223
15.6      224
13.9      234
16.7      252
12.8      264
14.1      298
14.8      317
14.2      346
14.0      354
14.4      427
13.1      437
1.7       552
11.9      573
13.7      579
13.4      627
12.9      719
12.7      733
13.3      863
12.3     1010
12.6     1044
11.6     1097
13.0     1267
11.7     1413
12.0     1456
12.4     1526
10.9     1613
12.5     1615
11.3     1772
12.2     1847
11.5     1859
11.8     1893
12.1     1942
11.0     2001
11.4     2134
11.2     2235
10.6     2357
11.1     2528
10.8     2679
9.8      2734
10.4     2749
10.3     2961
10.0     3273
10.7     3307
9.0      3422
9.7      3426
9.9      3504
10.2     3660
10.1     3728
8.9      3943
9.3      4019
10.5     4145
9.2      4305
9.5      4344
9.4      4443
9.1   

In [25]:
df.shape

(273453, 9)

In [26]:
mask_2= df["date_of_a1c"]== 1.7

In [27]:
mask_2.value_counts()

False    272901
True        552
Name: date_of_a1c, dtype: int64

In [28]:
def to_del(value_to_delete):
    to_del_array=[]
    for i in range(len(value_to_delete)):
        if value_to_delete[i]== True: 
            to_del_array.append(i)
    df.drop(to_del_array,inplace=True)
    df.reset_index(drop=True,inplace=True)

In [29]:
to_del(mask_2)

[59052, 59053, 59054, 59055, 59056, 59057, 59070, 59071, 59072, 59073, 59074, 59075, 59124, 59125, 59126, 59127, 59128, 59129, 59166, 59167, 59168, 59169, 59170, 59171, 59184, 59185, 59186, 59187, 59188, 59189, 59244, 59245, 59246, 59247, 59248, 59249, 59256, 59257, 59258, 59259, 59260, 59261, 59286, 59287, 59288, 59289, 59290, 59291, 59316, 59317, 59318, 59319, 59320, 59321, 59334, 59335, 59336, 59337, 59338, 59339, 59370, 59371, 59372, 59373, 59374, 59375, 59382, 59383, 59384, 59385, 59386, 59387, 59562, 59563, 59564, 59565, 59566, 59567, 59586, 59587, 59588, 59589, 59590, 59591, 59682, 59683, 59684, 59685, 59686, 59687, 59712, 59713, 59714, 59715, 59716, 59717, 59826, 59827, 59828, 59829, 59830, 59831, 59856, 59857, 59858, 59859, 59860, 59861, 59880, 59881, 59882, 59883, 59884, 59885, 59910, 59911, 59912, 59913, 59914, 59915, 59916, 59917, 59918, 59919, 59920, 59921, 59952, 59953, 59954, 59955, 59956, 59957, 59964, 59965, 59966, 59967, 59968, 59969, 60006, 60007, 60008, 60009, 60010

In [30]:
df.shape

(272901, 9)

In [ ]:
# df.drop(to_del_array,inplace=True)

In [ ]:
# df.shape

In [ ]:
# df.reset_index(drop=True,inplace=True)

In [31]:
df["date_of_a1c"].min()

7.0

In [32]:
df["date_of_a1c"].value_counts(ascending=True)

19.7        5
15.2       16
17.0       20
15.4       34
17.1       39
15.3       40
16.9       44
13.6       60
14.3      107
14.5      115
14.9      116
14.6      120
15.0      120
13.2      122
15.5      164
14.7      180
13.8      202
13.5      223
15.6      224
13.9      234
16.7      252
12.8      264
14.1      298
14.8      317
14.2      346
14.0      354
14.4      427
13.1      437
11.9      573
13.7      579
13.4      627
12.9      719
12.7      733
13.3      863
12.3     1010
12.6     1044
11.6     1097
13.0     1267
11.7     1413
12.0     1456
12.4     1526
10.9     1613
12.5     1615
11.3     1772
12.2     1847
11.5     1859
11.8     1893
12.1     1942
11.0     2001
11.4     2134
11.2     2235
10.6     2357
11.1     2528
10.8     2679
9.8      2734
10.4     2749
10.3     2961
10.0     3273
10.7     3307
9.0      3422
9.7      3426
9.9      3504
10.2     3660
10.1     3728
8.9      3943
9.3      4019
10.5     4145
9.2      4305
9.5      4344
9.4      4443
9.1      4475
9.6   

In [33]:
df["insurance"].value_counts()

Sliding Fee Scale                                62917
Medicaid - Husky D                               28659
Medicaid - Husky C                               24322
DO NOT USE - Medicaid State of CT                23376
National Government Services                     21913
Dental Sliding Fee                               17748
Medicaid - Husky A                               15554
Connecticare Exchange                             6944
DO NOT USE-Community Health Network - Husky A     6363
Medicaid State of CT                              6318
DO NOT USE - CHN Saga- Medicaid LIA               6174
Anthem Medicare                                   5742
United Healthcare Medicare                        4564
Connecticare                                      4546
CT Behavioral Health Partnership                  4216
Anthem BCBS                                       4010
Aetna Commercial                                  2675
Aetna Medicare -HMO                               2657
CT Dental 

In [36]:
df["visit_date"].head()

0    7/19/2017 14:30:00
1    7/19/2017 14:30:00
2    7/19/2017 14:30:00
3    7/19/2017 14:30:00
4    7/19/2017 14:30:00
Name: visit_date, dtype: object

In [39]:
df['visit_date'] = pd.to_datetime(df['visit_date'], format = '%')

In [69]:
df['obs_date'] = pd.to_datetime(df['obs_date'])

In [40]:
df.head()

index  patient_id  age           Birthdate          visit_date  \
0      0          72   56  11/26/1963 5:00:00 2017-07-19 14:30:00   
1      1          72   56  11/26/1963 5:00:00 2017-07-19 14:30:00   
2      2          72   56  11/26/1963 5:00:00 2017-07-19 14:30:00   
3      3          72   56  11/26/1963 5:00:00 2017-07-19 14:30:00   
4      4          72   56  11/26/1963 5:00:00 2017-07-19 14:30:00   

                         summary            obs_date  date_of_a1c  \
0  followup diabetes, med refill  7/21/2017 17:21:00          8.1   
1  followup diabetes, med refill  7/21/2017 17:21:00          8.1   
2  followup diabetes, med refill   1/19/2019 0:01:15          7.6   
3  followup diabetes, med refill   1/19/2019 0:01:15          7.6   
4  followup diabetes, med refill   2/8/2018 19:50:00          8.3   

            insurance  
0  Dental Sliding Fee  
1   Sliding Fee Scale  
2  Dental Sliding Fee  
3   Sliding Fee Scale  
4  Dental Sliding Fee

In [41]:
df.dtypes

index                   int64
patient_id              int64
age                     int64
Birthdate              object
visit_date     datetime64[ns]
summary                object
obs_date               object
date_of_a1c           float64
insurance              object
dtype: object

In [43]:
df["age"].value_counts()

59    20338
65    14184
64    13715
61    11974
69     9924
57     9588
73     8664
60     8546
62     7687
66     7466
67     7381
53     7284
68     7240
70     7198
50     6995
63     6883
54     6401
58     6380
49     6292
75     6180
74     5998
81     5774
56     5529
51     5397
55     4990
45     4950
47     4629
46     4384
52     4225
71     3676
87     3404
72     3306
79     2944
77     2675
41     2593
48     2389
80     1892
84     1808
44     1681
37     1515
34     1474
39     1463
78     1240
76     1222
38     1203
32     1193
90     1104
35      956
88      839
83      820
92      800
40      754
42      729
33      619
27      552
31      522
20      496
36      363
43      328
23      318
25      308
24      240
29      210
89      200
96      180
82      147
26       99
28       94
93       90
95       68
86       68
85       45
18       36
21       20
30       10
19        6
3         4
4         2
Name: age, dtype: int64

In [44]:
#3/27/45/67/
df.loc[df["age"]==3]

index  patient_id  age          Birthdate          visit_date  \
252373  252957      230569    3  3/16/2016 4:00:00 2018-06-27 10:44:00   
252374  252958      230569    3  3/16/2016 4:00:00 2018-06-27 10:44:00   
252375  252959      230569    3  3/16/2016 4:00:00 2018-06-21 14:18:00   
252376  252960      230569    3  3/16/2016 4:00:00 2018-06-21 14:18:00   

       summary            obs_date  date_of_a1c  \
252373     NaN  8/28/2017 15:33:46          7.7   
252374     NaN  8/28/2017 15:33:46          7.7   
252375     NaN  8/28/2017 15:33:46          7.7   
252376     NaN  8/28/2017 15:33:46          7.7   

                               insurance  
252373  CT Behavioral Health Partnership  
252374                 Sliding Fee Scale  
252375  CT Behavioral Health Partnership  
252376                 Sliding Fee Scale

In [45]:
#take patient_id and convert to dataframe/csv 
df.loc[df["patient_id"]==230569]

index  patient_id  age          Birthdate          visit_date  \
252373  252957      230569    3  3/16/2016 4:00:00 2018-06-27 10:44:00   
252374  252958      230569    3  3/16/2016 4:00:00 2018-06-27 10:44:00   
252375  252959      230569    3  3/16/2016 4:00:00 2018-06-21 14:18:00   
252376  252960      230569    3  3/16/2016 4:00:00 2018-06-21 14:18:00   

       summary            obs_date  date_of_a1c  \
252373     NaN  8/28/2017 15:33:46          7.7   
252374     NaN  8/28/2017 15:33:46          7.7   
252375     NaN  8/28/2017 15:33:46          7.7   
252376     NaN  8/28/2017 15:33:46          7.7   

                               insurance  
252373  CT Behavioral Health Partnership  
252374                 Sliding Fee Scale  
252375  CT Behavioral Health Partnership  
252376                 Sliding Fee Scale

In [46]:
df.loc[df["age"]==18]

index  patient_id  age          Birthdate          visit_date  \
43745  43745      111445   18  4/16/2001 4:00:00 2019-11-14 13:45:00   
43746  43746      111445   18  4/16/2001 4:00:00 2019-11-14 13:45:00   
43747  43747      111445   18  4/16/2001 4:00:00 2019-11-14 13:45:00   
43748  43748      111445   18  4/16/2001 4:00:00 2019-11-14 13:45:00   
43749  43749      111445   18  4/16/2001 4:00:00 2019-11-14 13:45:00   
43750  43750      111445   18  4/16/2001 4:00:00 2019-11-14 13:45:00   
43751  43751      111445   18  4/16/2001 4:00:00 2019-12-10 15:15:00   
43752  43752      111445   18  4/16/2001 4:00:00 2019-12-10 15:15:00   
43753  43753      111445   18  4/16/2001 4:00:00 2019-12-10 15:15:00   
43754  43754      111445   18  4/16/2001 4:00:00 2019-12-10 15:15:00   
43755  43755      111445   18  4/16/2001 4:00:00 2019-12-10 15:15:00   
43756  43756      111445   18  4/16/2001 4:00:00 2019-12-10 15:15:00   
43757  43757      111445   18  4/16/2001 4:00:00 2019-10-18 15:45:00   
43758  43758      111445   18  4/16/2001 4:00:00 2019-10-18 15:45:00   
43759  43759      111445   18  4/16/2001 4:00:00 2019-10-18 15:45:00   
43760  43760      111445   18  4/16/2001 4:00:00 2019-10-18 15:45:00   
43761  43761      111445   18  4/16/2001 4:00:00 2019-10-18 15:45:00   
43762  43762      111445   18  4/16/2001 4:00:00 2019-10-18 15:45:00   
43763  43763      111445   18  4/16/2001 4:00:00 2019-11-14 13:45:00   
43764  43764      111445   18  4/16/2001 4:00:00 2019-11-14 13:45:00   
43765  43765      111445   18  4/16/2001 4:00:00 2019-11-14 13:45:00   
43766  43766      111445   18  4/16/2001 4:00:00 2019-11-14 13:45:00   
43767  43767      111445   18  4/16/2001 4:00:00 2019-11-14 13:45:00   
43768  43768      111445   18  4/16/2001 4:00:00 2019-11-14 13:45:00   
43769  43769      111445   18  4/16/2001 4:00:00 2019-10-18 15:45:00   
43770  43770      111445   18  4/16/2001 4:00:00 2019-10-18 15:45:00   
43771  43771      111445   18  4/16/2001 4:00:00 2019-10-18 15:45:00   
43772  43772      111445   18  4/16/2001 4:00:00 2019-10-18 15:45:00   
43773  43773      111445   18  4/16/2001 4:00:00 2019-10-18 15:45:00   
43774  43774      111445   18  4/16/2001 4:00:00 2019-10-18 15:45:00   
43775  43775      111445   18  4/16/2001 4:00:00 2020-01-14 11:00:00   
43776  43776      111445   18  4/16/2001 4:00:00 2020-01-14 11:00:00   
43777  43777      111445   18  4/16/2001 4:00:00 2020-01-14 11:00:00   
43778  43778      111445   18  4/16/2001 4:00:00 2020-01-14 11:00:00   
43779  43779      111445   18  4/16/2001 4:00:00 2020-01-14 11:00:00   
43780  43780      111445   18  4/16/2001 4:00:00 2020-01-14 11:00:00   

                                                 summary             obs_date  \
43745  DM(type?)2 f/u, ref Nutrition, pt refuses med ...  12/10/2019 13:04:27   
43746  DM(type?)2 f/u, ref Nutrition, pt refuses med ...  12/10/2019 13:04:27   
43747  DM(type?)2 f/u, ref Nutrition, pt refuses med ...  12/10/2019 13:04:27   
43748  DM(type?)2 f/u, ref Nutrition, pt refuses med ...  12/10/2019 13:04:27   
43749  DM(type?)2 f/u, ref Nutrition, pt refuses med ...  12/10/2019 13:04:27   
43750  DM(type?)2 f/u, ref Nutrition, pt refuses med ...  12/10/2019 13:04:27   
43751  DM2 f/u (A1c 9.2), add trulicity, periorbital ...  12/10/2019 13:04:27   
43752  DM2 f/u (A1c 9.2), add trulicity, periorbital ...  12/10/2019 13:04:27   
43753  DM2 f/u (A1c 9.2), add trulicity, periorbital ...  12/10/2019 13:04:27   
43754  DM2 f/u (A1c 9.2), add trulicity, periorbital ...  12/10/2019 13:04:27   
43755  DM2 f/u (A1c 9.2), add trulicity, periorbital ...  12/10/2019 13:04:27   
43756  DM2 f/u (A1c 9.2), add trulicity, periorbital ...  12/10/2019 13:04:27   
43757              F/U diabetes (Type 1??), hospital F/U  12/10/2019 13:04:27   
43758              F/U diabetes (Type 1??), hospital F/U  12/10/2019 13:04:27   
43759              F/U diabetes (Type 1??), hospital F/U  12/10/2019 13:04:27   
43760              F/U diabetes (Type 1??), ho

In [47]:
df.loc[df["age"]==32]

index  patient_id  age         Birthdate          visit_date  \
84162    84714      123673   32  5/7/1987 4:00:00 2017-08-19 10:15:00   
84163    84715      123673   32  5/7/1987 4:00:00 2017-08-19 10:15:00   
84164    84716      123673   32  5/7/1987 4:00:00 2017-08-19 10:15:00   
84165    84717      123673   32  5/7/1987 4:00:00 2017-08-19 10:15:00   
84166    84718      123673   32  5/7/1987 4:00:00 2017-08-19 10:15:00   
...        ...         ...  ...               ...                 ...   
272764  273348      242762   32  8/7/1987 4:00:00 2019-08-30 14:45:00   
272765  273349      242762   32  8/7/1987 4:00:00 2019-08-30 14:45:00   
272766  273350      242762   32  8/7/1987 4:00:00 2019-08-30 14:45:00   
272767  273351      242762   32  8/7/1987 4:00:00 2019-09-27 12:00:00   
272768  273352      242762   32  8/7/1987 4:00:00 2019-09-27 12:00:00   

                                                  summary            obs_date  \
84162                                            diabetes  7/11/2017 14:17:00   
84163                                            diabetes  7/11/2017 14:17:00   
84164                                            diabetes  7/11/2017 14:17:00   
84165                                            diabetes  7/11/2017 14:17:00   
84166                                            diabetes  7/11/2017 14:17:00   
...                                                   ...                 ...   
272764                  DM, nausea, neuropathic foot pain  8/31/2019 17:08:37   
272765                  DM, nausea, neuropathic foot pain  8/31/2019 17:08:37   
272766                  DM, nausea, neuropathic foot pain  8/30/2019 15:06:34   
272767  FU on labs, DM, Neuropathy Follow up, trying t...  8/31/2019 17:08:37   
272768  FU on labs, DM, Neuropathy Follow up, trying t...  8/30/2019 15:06:34   

        date_of_a1c                            insurance  
84162           9.2                   Medicaid - Husky C  
84163           9.2         National Government Services  
84164           9.2    DO NOT USE - Medicaid State of CT  
84165           9.2                    Sliding Fee Scale  
84166           9.2  DO NOT USE - CHN Saga- Medicaid LIA  
...             ...                                  ...  
272764          7.9                Connecticare Exchange  
272765          7.9                Connecticare Exchange  
272766          7.9                Connecticare Exchange  
272767          7.9                Connecticare Exchange  
272768          7.9                Connecticare Exchange  

[1193 rows x 9 columns]

In [58]:
# #ad a column controlled and ucontrolled 
# df["ac1_level"]= [for i in range(df.shape[0]) "controlled" if df["date_of_a1c"]<=8 else "uncontrolled" ]
df["ac1_level"]=""

In [59]:
df.head()

index  patient_id  age           Birthdate          visit_date  \
0      0          72   56  11/26/1963 5:00:00 2017-07-19 14:30:00   
1      1          72   56  11/26/1963 5:00:00 2017-07-19 14:30:00   
2      2          72   56  11/26/1963 5:00:00 2017-07-19 14:30:00   
3      3          72   56  11/26/1963 5:00:00 2017-07-19 14:30:00   
4      4          72   56  11/26/1963 5:00:00 2017-07-19 14:30:00   

                         summary            obs_date  date_of_a1c  \
0  followup diabetes, med refill  7/21/2017 17:21:00          8.1   
1  followup diabetes, med refill  7/21/2017 17:21:00          8.1   
2  followup diabetes, med refill   1/19/2019 0:01:15          7.6   
3  followup diabetes, med refill   1/19/2019 0:01:15          7.6   
4  followup diabetes, med refill   2/8/2018 19:50:00          8.3   

            insurance ac1_level  
0  Dental Sliding Fee            
1   Sliding Fee Scale            
2  Dental Sliding Fee            
3   Sliding Fee Scale            
4  Dental Sliding Fee

In [60]:
for i in range(df.shape[0]):
    if df["date_of_a1c"][i] <=8:
        df["ac1_level"][i]="controlled"
    else: df["ac1_level"][i]="uncontrolled"

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


index  patient_id  age           Birthdate          visit_date  \
272896  273480      243644   58  12/20/1961 5:00:00 2019-12-10 14:40:00   
272897  273481      243644   58  12/20/1961 5:00:00 2019-12-11 13:45:00   
272898  273482      243644   58  12/20/1961 5:00:00 2019-12-11 13:45:00   
272899  273483      243644   58  12/20/1961 5:00:00 2020-01-10 13:45:00   
272900  273484      243644   58  12/20/1961 5:00:00 2020-01-10 13:45:00   

                                     summary             obs_date  \
272896             lost meds, f/u on DM, HTN  11/20/2019 21:13:21   
272897  f/u DM, HTN, HLD, flu and TDap given  11/20/2019 21:13:21   
272898  f/u DM, HTN, HLD, flu and TDap given    12/3/2019 8:24:35   
272899                    HTN & DM follow up    12/3/2019 8:24:35   
272900                    HTN & DM follow up  11/20/2019 21:13:21   

        date_of_a1c           insurance     ac1_level  
272896         13.4  Medicaid - Husky D  uncontrolled  
272897         13.4  Medicaid - Husky D  uncontrolled  
272898         12.2  Medicaid - Husky D  uncontrolled  
272899         12.2  Medicaid - Husky D  uncontrolled  
272900         13.4  Medicaid - Husky D  uncontrolled

In [63]:
df.drop(columns=["Birthdate"],inplace=True)

In [65]:
df.tail()

index  patient_id  age          visit_date  \
272896  273480      243644   58 2019-12-10 14:40:00   
272897  273481      243644   58 2019-12-11 13:45:00   
272898  273482      243644   58 2019-12-11 13:45:00   
272899  273483      243644   58 2020-01-10 13:45:00   
272900  273484      243644   58 2020-01-10 13:45:00   

                                     summary             obs_date  \
272896             lost meds, f/u on DM, HTN  11/20/2019 21:13:21   
272897  f/u DM, HTN, HLD, flu and TDap given  11/20/2019 21:13:21   
272898  f/u DM, HTN, HLD, flu and TDap given    12/3/2019 8:24:35   
272899                    HTN & DM follow up    12/3/2019 8:24:35   
272900                    HTN & DM follow up  11/20/2019 21:13:21   

        date_of_a1c           insurance     ac1_level  
272896         13.4  Medicaid - Husky D  uncontrolled  
272897         13.4  Medicaid - Husky D  uncontrolled  
272898         12.2  Medicaid - Husky D  uncontrolled  
272899         12.2  Medicaid - Husky D  uncontrolled  
272900         13.4  Medicaid - Husky D  uncontrolled

In [67]:
df.drop(columns=["index"],inplace=True)

In [70]:
df.dtypes

patient_id              int64
age                     int64
visit_date     datetime64[ns]
summary                object
obs_date       datetime64[ns]
date_of_a1c           float64
insurance              object
ac1_level              object
dtype: object

In [71]:
df.to_csv("../data/cleaned_data.csv")

In [4]:
new_data= pd.read_csv("../data/cleaned_data.csv")

In [5]:
#loop through visits and group number of visits 
new_data.drop(columns=["Unnamed: 0"],inplace=True)

VisitDate , summary, date of a1c has repeated values because the date when a1c was recorded and the visit date are not necessarily the same and patient has different visit dates and different date of a1c, basically its two data sets one would be patientid, summary and visit and the other would be Patientid, date of a1c and a1c_value, Combining those two will create duplicates.

In [6]:
new_data.loc[new_data["patient_id"]==72].sort_values(by=["visit_date","obs_date"])

patient_id  age           visit_date  \
0            72   56  2017-07-19 14:30:00   
1            72   56  2017-07-19 14:30:00   
6            72   56  2017-07-19 14:30:00   
7            72   56  2017-07-19 14:30:00   
4            72   56  2017-07-19 14:30:00   
5            72   56  2017-07-19 14:30:00   
2            72   56  2017-07-19 14:30:00   
3            72   56  2017-07-19 14:30:00   
8            72   56  2017-07-19 14:30:00   
9            72   56  2017-07-19 14:30:00   
10           72   56  2017-07-19 14:30:00   
11           72   56  2017-07-19 14:30:00   
22           72   56  2017-07-28 15:30:00   
23           72   56  2017-07-28 15:30:00   
18           72   56  2017-07-28 15:30:00   
19           72   56  2017-07-28 15:30:00   
16           72   56  2017-07-28 15:30:00   
17           72   56  2017-07-28 15:30:00   
20           72   56  2017-07-28 15:30:00   
21           72   56  2017-07-28 15:30:00   
14           72   56  2017-07-28 15:30:00   
15           72   56  2017-07-28 15:30:00   
12           72   56  2017-07-28 15:30:00   
13           72   56  2017-07-28 15:30:00   
24           72   56  2017-11-08 14:30:00   
25           72   56  2017-11-08 14:30:00   
30           72   56  2017-11-08 14:30:00   
31           72   56  2017-11-08 14:30:00   
28           72   56  2017-11-08 14:30:00   
29           72   56  2017-11-08 14:30:00   
26           72   56  2017-11-08 14:30:00   
27           72   56  2017-11-08 14:30:00   
34           72   56  2017-11-08 14:30:00   
35           72   56  2017-11-08 14:30:00   
32           72   56  2017-11-08 14:30:00   
33           72   56  2017-11-08 14:30:00   
40           72   56  2018-02-16 20:27:00   
41           72   56  2018-02-16 20:27:00   
42           72   56  2018-02-16 20:27:00   
43           72   56  2018-02-16 20:27:00   
44           72   56  2018-02-16 20:27:00   
45           72   56  2018-02-16 20:27:00   
46           72   56  2018-02-16 20:27:00   
47           72   56  2018-02-16 20:27:00   
38           72   56  2018-02-16 20:27:00   
39           72   56  2018-02-16 20:27:00   
36           72   56  2018-02-16 20:27:00   
37           72   56  2018-02-16 20:27:00   
54           72   56  2018-05-02 14:30:00   
55           72   56  2018-05-02 14:30:00   
48           72   56  2018-05-02 14:30:00   
49           72   56  2018-05-02 14:30:00   
50           72   56  2018-05-02 14:30:00   
51           72   56  2018-05-02 14:30:00   
52           72   56  2018-05-02 14:30:00   
53           72   56  2018-05-02 14:30:00   
56           72   56  2018-05-02 14:30:00   
57           72   56  2018-05-02 14:30:00   
58           72   56  2018-05-02 14:30:00   
59           72   56  2018-05-02 14:30:00   
64           72   56  2018-09-28 14:00:00   
65           72   56  2018-09-28 14:00:00   
66           72   56  2018-09-28 14:00:00   
67           72   56  2018-09-28 14:00:00   
68           72   56  2018-09-28 14:00:00   
69           72   56  2018-09-28 14:00:00   
70           72   56  2018-09-28 14:00:00   
71           72   56  2018-09-28 14:00:00   
62           72   56  2018-09-28 14:00:00   
63           72   56  2018-09-28 14:00:00   
60           72   56  2018-09-28 14:00:00   
61           72   56  2018-09-28 14:00:00   
78           72   56  2019-01-18 13:45:00   
79           72   56  2019-01-18 13:45:00   
72           72   56  2019-01-18 13:45:00   
73           72   56  2019-01-18 13:45:00   
74           72   56  2019-01-18 13:45:00   
75           72   56  2019-01-18 13:45:00   
76           72   56  2019-01-18 13:45:00   
77           72   56  2019-01-18 13:45:00   
114          72   56  2019-01-18 13:45:00   
115          72   56  2019-01-18 13:45:00   
116          72   56  2019-01-18 13:45:00   
117          72   56  2019-01-18 13:45:00   
80           72   56  2019-05-17 14:15:00   
81           72   56  2019-05-17 14:15:00   
82           72   56  2019-05-17 14:15:00   
83           72   56  2019-05-17 14:15:00   


In [7]:
new_data.loc[new_data["patient_id"]==72].sort_values(by=["visit_date","obs_date"]).shape

(132, 8)

In [8]:
w_dups = new_data.loc[new_data["patient_id"]==72].sort_values(by=["visit_date","obs_date"])
w_dups

patient_id  age           visit_date  \
0            72   56  2017-07-19 14:30:00   
1            72   56  2017-07-19 14:30:00   
6            72   56  2017-07-19 14:30:00   
7            72   56  2017-07-19 14:30:00   
4            72   56  2017-07-19 14:30:00   
5            72   56  2017-07-19 14:30:00   
2            72   56  2017-07-19 14:30:00   
3            72   56  2017-07-19 14:30:00   
8            72   56  2017-07-19 14:30:00   
9            72   56  2017-07-19 14:30:00   
10           72   56  2017-07-19 14:30:00   
11           72   56  2017-07-19 14:30:00   
22           72   56  2017-07-28 15:30:00   
23           72   56  2017-07-28 15:30:00   
18           72   56  2017-07-28 15:30:00   
19           72   56  2017-07-28 15:30:00   
16           72   56  2017-07-28 15:30:00   
17           72   56  2017-07-28 15:30:00   
20           72   56  2017-07-28 15:30:00   
21           72   56  2017-07-28 15:30:00   
14           72   56  2017-07-28 15:30:00   
15           72   56  2017-07-28 15:30:00   
12           72   56  2017-07-28 15:30:00   
13           72   56  2017-07-28 15:30:00   
24           72   56  2017-11-08 14:30:00   
25           72   56  2017-11-08 14:30:00   
30           72   56  2017-11-08 14:30:00   
31           72   56  2017-11-08 14:30:00   
28           72   56  2017-11-08 14:30:00   
29           72   56  2017-11-08 14:30:00   
26           72   56  2017-11-08 14:30:00   
27           72   56  2017-11-08 14:30:00   
34           72   56  2017-11-08 14:30:00   
35           72   56  2017-11-08 14:30:00   
32           72   56  2017-11-08 14:30:00   
33           72   56  2017-11-08 14:30:00   
40           72   56  2018-02-16 20:27:00   
41           72   56  2018-02-16 20:27:00   
42           72   56  2018-02-16 20:27:00   
43           72   56  2018-02-16 20:27:00   
44           72   56  2018-02-16 20:27:00   
45           72   56  2018-02-16 20:27:00   
46           72   56  2018-02-16 20:27:00   
47           72   56  2018-02-16 20:27:00   
38           72   56  2018-02-16 20:27:00   
39           72   56  2018-02-16 20:27:00   
36           72   56  2018-02-16 20:27:00   
37           72   56  2018-02-16 20:27:00   
54           72   56  2018-05-02 14:30:00   
55           72   56  2018-05-02 14:30:00   
48           72   56  2018-05-02 14:30:00   
49           72   56  2018-05-02 14:30:00   
50           72   56  2018-05-02 14:30:00   
51           72   56  2018-05-02 14:30:00   
52           72   56  2018-05-02 14:30:00   
53           72   56  2018-05-02 14:30:00   
56           72   56  2018-05-02 14:30:00   
57           72   56  2018-05-02 14:30:00   
58           72   56  2018-05-02 14:30:00   
59           72   56  2018-05-02 14:30:00   
64           72   56  2018-09-28 14:00:00   
65           72   56  2018-09-28 14:00:00   
66           72   56  2018-09-28 14:00:00   
67           72   56  2018-09-28 14:00:00   
68           72   56  2018-09-28 14:00:00   
69           72   56  2018-09-28 14:00:00   
70           72   56  2018-09-28 14:00:00   
71           72   56  2018-09-28 14:00:00   
62           72   56  2018-09-28 14:00:00   
63           72   56  2018-09-28 14:00:00   
60           72   56  2018-09-28 14:00:00   
61           72   56  2018-09-28 14:00:00   
78           72   56  2019-01-18 13:45:00   
79           72   56  2019-01-18 13:45:00   
72           72   56  2019-01-18 13:45:00   
73           72   56  2019-01-18 13:45:00   
74           72   56  2019-01-18 13:45:00   
75           72   56  2019-01-18 13:45:00   
76           72   56  2019-01-18 13:45:00   
77           72   56  2019-01-18 13:45:00   
114          72   56  2019-01-18 13:45:00   
115          72   56  2019-01-18 13:45:00   
116          72   56  2019-01-18 13:45:00   
117          72   56  2019-01-18 13:45:00   
80           72   56  2019-05-17 14:15:00   
81           72   56  2019-05-17 14:15:00   
82           72   56  2019-05-17 14:15:00   
83           72   56  2019-05-17 14:15:00   


In [9]:
no_dups = new_data.loc[new_data["patient_id"]==72].sort_values(by=["visit_date","obs_date"]).drop_duplicates()
no_dups

patient_id  age           visit_date  \
0            72   56  2017-07-19 14:30:00   
1            72   56  2017-07-19 14:30:00   
6            72   56  2017-07-19 14:30:00   
7            72   56  2017-07-19 14:30:00   
4            72   56  2017-07-19 14:30:00   
5            72   56  2017-07-19 14:30:00   
2            72   56  2017-07-19 14:30:00   
3            72   56  2017-07-19 14:30:00   
8            72   56  2017-07-19 14:30:00   
9            72   56  2017-07-19 14:30:00   
10           72   56  2017-07-19 14:30:00   
11           72   56  2017-07-19 14:30:00   
22           72   56  2017-07-28 15:30:00   
23           72   56  2017-07-28 15:30:00   
18           72   56  2017-07-28 15:30:00   
19           72   56  2017-07-28 15:30:00   
16           72   56  2017-07-28 15:30:00   
17           72   56  2017-07-28 15:30:00   
20           72   56  2017-07-28 15:30:00   
21           72   56  2017-07-28 15:30:00   
14           72   56  2017-07-28 15:30:00   
15           72   56  2017-07-28 15:30:00   
12           72   56  2017-07-28 15:30:00   
13           72   56  2017-07-28 15:30:00   
24           72   56  2017-11-08 14:30:00   
25           72   56  2017-11-08 14:30:00   
30           72   56  2017-11-08 14:30:00   
31           72   56  2017-11-08 14:30:00   
28           72   56  2017-11-08 14:30:00   
29           72   56  2017-11-08 14:30:00   
26           72   56  2017-11-08 14:30:00   
27           72   56  2017-11-08 14:30:00   
34           72   56  2017-11-08 14:30:00   
35           72   56  2017-11-08 14:30:00   
32           72   56  2017-11-08 14:30:00   
33           72   56  2017-11-08 14:30:00   
40           72   56  2018-02-16 20:27:00   
41           72   56  2018-02-16 20:27:00   
42           72   56  2018-02-16 20:27:00   
43           72   56  2018-02-16 20:27:00   
44           72   56  2018-02-16 20:27:00   
45           72   56  2018-02-16 20:27:00   
46           72   56  2018-02-16 20:27:00   
47           72   56  2018-02-16 20:27:00   
38           72   56  2018-02-16 20:27:00   
39           72   56  2018-02-16 20:27:00   
36           72   56  2018-02-16 20:27:00   
37           72   56  2018-02-16 20:27:00   
54           72   56  2018-05-02 14:30:00   
55           72   56  2018-05-02 14:30:00   
48           72   56  2018-05-02 14:30:00   
49           72   56  2018-05-02 14:30:00   
50           72   56  2018-05-02 14:30:00   
51           72   56  2018-05-02 14:30:00   
52           72   56  2018-05-02 14:30:00   
53           72   56  2018-05-02 14:30:00   
56           72   56  2018-05-02 14:30:00   
57           72   56  2018-05-02 14:30:00   
58           72   56  2018-05-02 14:30:00   
59           72   56  2018-05-02 14:30:00   
64           72   56  2018-09-28 14:00:00   
65           72   56  2018-09-28 14:00:00   
66           72   56  2018-09-28 14:00:00   
67           72   56  2018-09-28 14:00:00   
68           72   56  2018-09-28 14:00:00   
69           72   56  2018-09-28 14:00:00   
70           72   56  2018-09-28 14:00:00   
71           72   56  2018-09-28 14:00:00   
62           72   56  2018-09-28 14:00:00   
63           72   56  2018-09-28 14:00:00   
60           72   56  2018-09-28 14:00:00   
61           72   56  2018-09-28 14:00:00   
78           72   56  2019-01-18 13:45:00   
79           72   56  2019-01-18 13:45:00   
72           72   56  2019-01-18 13:45:00   
73           72   56  2019-01-18 13:45:00   
74           72   56  2019-01-18 13:45:00   
75           72   56  2019-01-18 13:45:00   
76           72   56  2019-01-18 13:45:00   
77           72   56  2019-01-18 13:45:00   
114          72   56  2019-01-18 13:45:00   
115          72   56  2019-01-18 13:45:00   
116          72   56  2019-01-18 13:45:00   
117          72   56  2019-01-18 13:45:00   
80           72   56  2019-05-17 14:15:00   
81           72   56  2019-05-17 14:15:00   
82           72   56  2019-05-17 14:15:00   
83           72   56  2019-05-17 14:15:00   


In [10]:
w_dups.loc[w_dups.duplicated() == True]

patient_id  age           visit_date  \
110          72   56  2020-01-03 09:30:00   
111          72   56  2020-01-03 09:30:00   
108          72   56  2020-01-03 09:30:00   
109          72   56  2020-01-03 09:30:00   
104          72   56  2020-01-03 09:30:00   
105          72   56  2020-01-03 09:30:00   
106          72   56  2020-01-03 09:30:00   
107          72   56  2020-01-03 09:30:00   
130          72   56  2020-01-03 09:30:00   
131          72   56  2020-01-03 09:30:00   
128          72   56  2020-01-03 09:30:00   
129          72   56  2020-01-03 09:30:00   

                                              summary             obs_date  \
110  Worse a1c 8.2 from 7, no new meds, repeat lipids  2017-07-21 17:21:00   
111  Worse a1c 8.2 from 7, no new meds, repeat lipids  2017-07-21 17:21:00   
108  Worse a1c 8.2 from 7, no new meds, repeat lipids  2017-10-20 17:54:00   
109  Worse a1c 8.2 from 7, no new meds, repeat lipids  2017-10-20 17:54:00   
104  Worse a1c 8.2 from 7, no new meds, repeat lipids  2018-02-08 19:50:00   
105  Worse a1c 8.2 from 7, no new meds, repeat lipids  2018-02-08 19:50:00   
106  Worse a1c 8.2 from 7, no new meds, repeat lipids  2019-01-19 00:01:15   
107  Worse a1c 8.2 from 7, no new meds, repeat lipids  2019-01-19 00:01:15   
130  Worse a1c 8.2 from 7, no new meds, repeat lipids  2019-06-08 11:34:19   
131  Worse a1c 8.2 from 7, no new meds, repeat lipids  2019-06-08 11:34:19   
128  Worse a1c 8.2 from 7, no new meds, repeat lipids  2019-12-09 14:36:12   
129  Worse a1c 8.2 from 7, no new meds, repeat lipids  2019-12-09 14:36:12   

     date_of_a1c           insurance     ac1_level  
110          8.1  Dental Sliding Fee  uncontrolled  
111          8.1   Sliding Fee Scale  uncontrolled  
108          7.7  Dental Sliding Fee    controlled  
109          7.7   Sliding Fee Scale    controlled  
104          8.3  Dental Sliding Fee  uncontrolled  
105          8.3   Sliding Fee Scale  uncontrolled  
106          7.6  Dental Sliding Fee    controlled  
107          7.6   Sliding Fee Scale    controlled  
130          7.1  Dental Sliding Fee    controlled  
131          7.1   Sliding Fee Scale    controlled  
128          8.2  Dental Sliding Fee  uncontrolled  
129          8.2   Sliding Fee Scale  uncontrolled

In [11]:
#create two tables 
#s one would be patientid, summary and visit 
# the other would be Patientid, date of a1c and a1c_value,

#frequnecy of getting blood drawn and frequency of visit? 
#is there a way to  say/ if obs date is within a week of visit date, 


summary_and_visit_df = new_data[['patient_id', 'age', 'visit_date','summary','insurance']].copy()
date_a1c_df=new_data[['patient_id', 'age', 'obs_date','insurance','date_of_a1c','ac1_level']].copy()

In [12]:
summary_and_visit_df.head()

patient_id  age           visit_date                        summary  \
0          72   56  2017-07-19 14:30:00  followup diabetes, med refill   
1          72   56  2017-07-19 14:30:00  followup diabetes, med refill   
2          72   56  2017-07-19 14:30:00  followup diabetes, med refill   
3          72   56  2017-07-19 14:30:00  followup diabetes, med refill   
4          72   56  2017-07-19 14:30:00  followup diabetes, med refill   

            insurance  
0  Dental Sliding Fee  
1   Sliding Fee Scale  
2  Dental Sliding Fee  
3   Sliding Fee Scale  
4  Dental Sliding Fee

In [13]:
summary_and_visit_df.shape[0] == date_a1c_df.shape[0]

True

In [14]:
date_a1c_df.head()
#different insurances 

patient_id  age             obs_date           insurance  date_of_a1c  \
0          72   56  2017-07-21 17:21:00  Dental Sliding Fee          8.1   
1          72   56  2017-07-21 17:21:00   Sliding Fee Scale          8.1   
2          72   56  2019-01-19 00:01:15  Dental Sliding Fee          7.6   
3          72   56  2019-01-19 00:01:15   Sliding Fee Scale          7.6   
4          72   56  2018-02-08 19:50:00  Dental Sliding Fee          8.3   

      ac1_level  
0  uncontrolled  
1  uncontrolled  
2    controlled  
3    controlled  
4  uncontrolled

In [15]:
date_a1c_df.shape

(272901, 6)

In [16]:
date_a1c_df.shape[0] - summary_and_visit_df.drop_duplicates().shape[0]

216889

In [17]:
new_data.loc[new_data['patient_id'] == 243644]

patient_id  age           visit_date  \
272891      243644   58  2019-11-20 13:45:00   
272892      243644   58  2019-11-20 13:45:00   
272893      243644   58  2019-11-27 16:59:00   
272894      243644   58  2019-11-27 16:59:00   
272895      243644   58  2019-12-10 14:40:00   
272896      243644   58  2019-12-10 14:40:00   
272897      243644   58  2019-12-11 13:45:00   
272898      243644   58  2019-12-11 13:45:00   
272899      243644   58  2020-01-10 13:45:00   
272900      243644   58  2020-01-10 13:45:00   

                                                  summary  \
272891  establish care, new diabetic on lantus and met...   
272892  establish care, new diabetic on lantus and met...   
272893                                Diabetes ref letter   
272894                                Diabetes ref letter   
272895                          lost meds, f/u on DM, HTN   
272896                          lost meds, f/u on DM, HTN   
272897               f/u DM, HTN, HLD, flu and TDap given   
272898               f/u DM, HTN, HLD, flu and TDap given   
272899                                 HTN & DM follow up   
272900                                 HTN & DM follow up   

                   obs_date  date_of_a1c           insurance     ac1_level  
272891  2019-11-20 21:13:21         13.4  Medicaid - Husky D  uncontrolled  
272892  2019-12-03 08:24:35         12.2  Medicaid - Husky D  uncontrolled  
272893  2019-11-20 21:13:21         13.4  Medicaid - Husky D  uncontrolled  
272894  2019-12-03 08:24:35         12.2  Medicaid - Husky D  uncontrolled  
272895  2019-12-03 08:24:35         12.2  Medicaid - Husky D  uncontrolled  
272896  2019-11-20 21:13:21         13.4  Medicaid - Husky D  uncontrolled  
272897  2019-11-20 21:13:21         13.4  Medicaid - Husky D  uncontrolled  
272898  2019-12-03 08:24:35         12.2  Medicaid - Husky D  uncontrolled  
272899  2019-12-03 08:24:35         12.2  Medicaid - Husky D  uncontrolled  
272900  2019-11-20 21:13:21         13.4  Medicaid - Husky D  uncontrolled

In [18]:
#if patiend id is the same and obs date no more than a week later, then associat ethose two visits together/ add column 
#frequency of visits - correlation between frequency and a1c 
#get rid of time 
# count frequnecy of visits/year - EDA with a1c levels that year since 2017 
#limitations and future steps 

In [19]:
summary_and_visit_df.shape

(272901, 5)

In [20]:
summary_and_visit_df.drop_duplicates(inplace=True)

In [21]:
summary_and_visit_df.shape

(56012, 5)

In [22]:
date_a1c_df.shape

(272901, 6)

In [23]:
date_a1c_df.drop_duplicates(inplace=True)

In [24]:
date_a1c_df.shape

(13703, 6)

In [25]:
date_a1c_df.head()

patient_id  age             obs_date           insurance  date_of_a1c  \
0          72   56  2017-07-21 17:21:00  Dental Sliding Fee          8.1   
1          72   56  2017-07-21 17:21:00   Sliding Fee Scale          8.1   
2          72   56  2019-01-19 00:01:15  Dental Sliding Fee          7.6   
3          72   56  2019-01-19 00:01:15   Sliding Fee Scale          7.6   
4          72   56  2018-02-08 19:50:00  Dental Sliding Fee          8.3   

      ac1_level  
0  uncontrolled  
1  uncontrolled  
2    controlled  
3    controlled  
4  uncontrolled

In [26]:
date_a1c_df["patient_id"].value_counts()

971       105
386        99
203824     88
1158       77
209388     77
         ... 
234136      1
218095      1
242919      1
234253      1
241838      1
Name: patient_id, Length: 1099, dtype: int64

In [27]:
summary_and_visit_df

patient_id  age           visit_date  \
0               72   56  2017-07-19 14:30:00   
1               72   56  2017-07-19 14:30:00   
12              72   56  2017-07-28 15:30:00   
13              72   56  2017-07-28 15:30:00   
24              72   56  2017-11-08 14:30:00   
...            ...  ...                  ...   
272891      243644   58  2019-11-20 13:45:00   
272893      243644   58  2019-11-27 16:59:00   
272895      243644   58  2019-12-10 14:40:00   
272897      243644   58  2019-12-11 13:45:00   
272899      243644   58  2020-01-10 13:45:00   

                                                  summary           insurance  
0                           followup diabetes, med refill  Dental Sliding Fee  
1                           followup diabetes, med refill   Sliding Fee Scale  
12                              f/u lab results, DM2, HLD  Dental Sliding Fee  
13                              f/u lab results, DM2, HLD   Sliding Fee Scale  
24                          DM and dyslipidemia follow-up  Dental Sliding Fee  
...                                                   ...                 ...  
272891  establish care, new diabetic on lantus and met...  Medicaid - Husky D  
272893                                Diabetes ref letter  Medicaid - Husky D  
272895                          lost meds, f/u on DM, HTN  Medicaid - Husky D  
272897               f/u DM, HTN, HLD, flu and TDap given  Medicaid - Husky D  
272899                                 HTN & DM follow up  Medicaid - Husky D  

[56012 rows x 5 columns]

In [34]:
date_a1c_df.shape

(13703, 6)

In [58]:
summary_and_visit_df.shape

(56012, 5)

In [61]:
date_a1c_df.shape[0] + summary_and_visit_df.shape[0]

69715

In [29]:
date_a1c_df['obs_date'] = pd.to_datetime(date_a1c_df['obs_date'])

In [30]:
summary_and_visit_df['visit_date'] = pd.to_datetime(summary_and_visit_df['visit_date'])

In [62]:
date_a1c_df['obs_date'] - summary_and_visit_df['visit_date']

0        2 days 02:51:00
1        2 days 02:51:00
2                    NaT
3                    NaT
4                    NaT
               ...      
272892               NaT
272893               NaT
272895               NaT
272897               NaT
272899               NaT
Length: 61839, dtype: timedelta64[ns]

In [65]:
date_a1c_df['obs_date'][0] - summary_and_visit_df['visit_date'][0]

datetime.timedelta(days=2)

In [56]:
(date_a1c_df['obs_date'] - summary_and_visit_df['visit_date'])

Timedelta('-1059 days +02:06:00')

In [64]:
# Making values dates only, no time
date_a1c_df['obs_date'] = date_a1c_df['obs_date'].dt.date
summary_and_visit_df['visit_date'] = summary_and_visit_df['visit_date'].dt.date

In [66]:
date_a1c_df.head()

patient_id  age    obs_date           insurance  date_of_a1c     ac1_level
0          72   56  2017-07-21  Dental Sliding Fee          8.1  uncontrolled
1          72   56  2017-07-21   Sliding Fee Scale          8.1  uncontrolled
2          72   56  2019-01-19  Dental Sliding Fee          7.6    controlled
3          72   56  2019-01-19   Sliding Fee Scale          7.6    controlled
4          72   56  2018-02-08  Dental Sliding Fee          8.3  uncontrolled

In [67]:
summary_and_visit_df.head()

patient_id  age  visit_date                        summary  \
0           72   56  2017-07-19  followup diabetes, med refill   
1           72   56  2017-07-19  followup diabetes, med refill   
12          72   56  2017-07-28      f/u lab results, DM2, HLD   
13          72   56  2017-07-28      f/u lab results, DM2, HLD   
24          72   56  2017-11-08  DM and dyslipidemia follow-up   

             insurance  
0   Dental Sliding Fee  
1    Sliding Fee Scale  
12  Dental Sliding Fee  
13   Sliding Fee Scale  
24  Dental Sliding Fee

In [ ]:
for _ in date_alc_df['patient_id'].unique():

In [98]:
fooo = date_a1c_df.loc[date_a1c_df['patient_id'] == 72]
barr = summary_and_visit_df.loc[summary_and_visit_df['patient_id'] == 72]
fooo["obs_date"].nunique()

6

In [99]:
fooo

patient_id  age    obs_date           insurance  date_of_a1c     ac1_level
0           72   56  2017-07-21  Dental Sliding Fee          8.1  uncontrolled
1           72   56  2017-07-21   Sliding Fee Scale          8.1  uncontrolled
2           72   56  2019-01-19  Dental Sliding Fee          7.6    controlled
3           72   56  2019-01-19   Sliding Fee Scale          7.6    controlled
4           72   56  2018-02-08  Dental Sliding Fee          8.3  uncontrolled
5           72   56  2018-02-08   Sliding Fee Scale          8.3  uncontrolled
6           72   56  2017-10-20  Dental Sliding Fee          7.7    controlled
7           72   56  2017-10-20   Sliding Fee Scale          7.7    controlled
8           72   56  2019-06-08  Dental Sliding Fee          7.1    controlled
9           72   56  2019-06-08   Sliding Fee Scale          7.1    controlled
10          72   56  2019-12-09  Dental Sliding Fee          8.2  uncontrolled
11          72   56  2019-12-09   Sliding Fee Scale          8.2  uncontrolled

In [78]:
poo = date_a1c_df['obs_date'] - summary_and_visit_df['visit_date']

In [95]:
poo.loc[poo.isna() == False]

0          2 days
1          2 days
132      527 days
133      527 days
134      527 days
           ...   
272876     0 days
272877     0 days
272887     1 days
272888     1 days
272891     0 days
Length: 7876, dtype: timedelta64[ns]

In [ ]:
def name_this_later(df0 = date_a1c_df, df1 = summary_and_visit_df):
    pooo = df0['obs_date'] - df1['visit_date']
    

In [96]:
poo.loc[poo.isna() == False].map(lambda diff: True if (diff < np.timedelta64(1,'W')) else False)

0          True
1          True
132       False
133       False
134       False
          ...  
272876     True
272877     True
272887     True
272888     True
272891     True
Length: 7876, dtype: bool

In [88]:
poo.isna()

0         False
1         False
2          True
3          True
4          True
          ...  
272892     True
272893     True
272895     True
272897     True
272899     True
Length: 61839, dtype: bool

In [ ]:
fooo.map(lambda obs_date: True if (barr['visit_date'] - obs_date) == )

In [ ]:
def check_daterange(patient_id, df0 = date_a1c_df, col0 = 'obs_date', df1 = summary_and_visit_df, col1 = 'visit_date'):
    foo = df0.loc[df0['patient_id'] == patient_id]
    bar = df1.loc[df1['patient_id'] == patient_id]
    

In [32]:
date_a1c_df['visit_date']=""
date_a1c_df['summary'] = "" 
date_a1c_df['a1c_level'] = "" 

def visit_date(obs_date,visit_date,summary,a1c_level):
    if obs_date - visit_date <= 8 
    date_a1c_df['visit_date'] = visit_date 
    date_a1c_df['summary'] = summary
    
    

SyntaxError: invalid syntax (<ipython-input-32-b41604242462>, line 1)

In [ ]:
for index, row in date_a1c_df.iterrows():
    if (date_a1c_df['obs_date'][index] - summary_and_visit_df['visit_date'][]index)

In [105]:
date_a1c_df['patient_id'][0]

72

In [112]:
#num obs date and num visit date
#count unique values with patient id
date_a1c_df["num_obs"]=""

for i in range(len(date_a1c_df['obs_date'])):
    try:
        mas=date_a1c_df.loc[date_a1c_df['patient_id'] == date_a1c_df['patient_id'][i]]
        date_a1c_df["num_obs"][i]= mas['obs_date'].nunique()
    except: 
        continue 

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [118]:
summary_and_visit_df["num_visits"]=""

for i in range(len(summary_and_visit_df["num_visits"])):
    try:
        mas=summary_and_visit_df.loc[summary_and_visit_df['patient_id'] == summary_and_visit_df['patient_id'][i]]
        summary_and_visit_df["num_visits"][i]= mas["visit_date"].nunique()
    except: 
        continue

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [130]:
date_a1c_df.head()

patient_id  age    obs_date           insurance  date_of_a1c     ac1_level  \
0          72   56  2017-07-21  Dental Sliding Fee          8.1  uncontrolled   
1          72   56  2017-07-21   Sliding Fee Scale          8.1  uncontrolled   
2          72   56  2019-01-19  Dental Sliding Fee          7.6    controlled   
3          72   56  2019-01-19   Sliding Fee Scale          7.6    controlled   
4          72   56  2018-02-08  Dental Sliding Fee          8.3  uncontrolled   

  num_obs  
0       6  
1       6  
2       6  
3       6  
4       6

In [129]:
summary_and_visit_df.head()

patient_id  age  visit_date                        summary  \
0           72   56  2017-07-19  followup diabetes, med refill   
1           72   56  2017-07-19  followup diabetes, med refill   
12          72   56  2017-07-28      f/u lab results, DM2, HLD   
13          72   56  2017-07-28      f/u lab results, DM2, HLD   
24          72   56  2017-11-08  DM and dyslipidemia follow-up   

             insurance num_visits  
0   Dental Sliding Fee         10  
1    Sliding Fee Scale         10  
12  Dental Sliding Fee         10  
13   Sliding Fee Scale         10  
24  Dental Sliding Fee         10

In [125]:
#add blank columsn to summary and visit 
summary_and_visit_df["a1c_level_num"]=""
summary_and_visit_df["a1c_level"]=""

for i in range(summary_and_visit_df.shape[0]):
    
    if date_a1c_df['obs_date'][i] - summary_and_visit_df['visit_date'][i] <= np.timedelta64(1,'W')
    

patient_id           age
count   56012.000000  56012.000000
mean   161948.825323     60.961580
std     71561.619721     12.407278
min        72.000000      3.000000
25%    121038.000000     53.000000
50%    202947.000000     61.000000
75%    218386.000000     69.000000
max    243644.000000     96.000000

In [ ]:
#date_a1c_df add number of visits

summary_and_visit_df["num_visits"]=""

for i in range(len(summary_and_visit_df["num_visits"])):
    try:
        mas=summary_and_visit_df.loc[summary_and_visit_df['patient_id'] == summary_and_visit_df['patient_id'][i]]
        summary_and_visit_df["num_visits"][i]= mas["visit_date"].nunique()
    except: 
        continue
        
        
        #add visit date and summary to date_a1c_df 

In [ ]:
date_a1c_df["visit_date"]=""
date_a1c_df["summary"]=""

for i in range(len(summary_and_visit_df)):
    try:
        mas=date_a1c_df.loc[date_a1c_df['patient_id'] == date_a1c_df['patient_id'][i]]
        mas_2 = summary_and_visit_df.loc[summary_and_visit_df['patient_id'] == summary_and_visit_df['patient_id'][i]]
        
        if date_a1c_df['obs_date'][i] - summary_and_visit_df['visit_date'][i] > np.timedelta64(1,'W')
        
    except: 
        continue 

In [133]:
df_summary= date_a1c_df.drop(columns=["insurance"])

In [142]:
df_a1c.shape

(14197, 5)

In [135]:
df_a1c = summary_and_visit_df.drop(columns=["insurance"])

In [138]:
df_summary.drop_duplicates(inplace=True)

In [144]:
df_a1c.drop_duplicates(inplace=True)

In [149]:
df_summary.head()

patient_id  age  visit_date                            summary num_visits
0           72   56  2017-07-19      followup diabetes, med refill         10
12          72   56  2017-07-28          f/u lab results, DM2, HLD         10
24          72   56  2017-11-08      DM and dyslipidemia follow-up         10
36          72   56  2018-02-16                    Uncontrolled DM         10
48          72   56  2018-05-02  DM - hypoglycemia- dec. glyburide         10

In [150]:
df_summary.shape

(14197, 5)

In [151]:
df_a1c.head()

patient_id  age    obs_date  date_of_a1c     ac1_level num_obs
0          72   56  2017-07-21          8.1  uncontrolled       6
2          72   56  2019-01-19          7.6    controlled       6
4          72   56  2018-02-08          8.3  uncontrolled       6
6          72   56  2017-10-20          7.7    controlled       6
8          72   56  2019-06-08          7.1    controlled       6

In [152]:
df_a1c.shape

(3617, 6)

In [166]:
np.unique(df_summary["patient_id"].values)

array([    72,    254,    255, ..., 243548, 243642, 243644])

In [170]:
for i in range(df_summary.shape[0]):
    for j in range(df_a1c.shape[0]):
        try: 
            if df_summary["patient_id"][i]==df_a1c["patient_id"][j]: 
                if df_a1c["obs_date"][j] - df_summary["visit_date"][i] <= np.timedelta64(2,'W'): 
                    df_summary["a1c_type"][i] = df_a1c["date_of_a1c"][j]
                    df_summary["a1c_level"][i]= df_a1c["ac1_level"][j]
        except:
            continue

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


KeyboardInterrupt: 

In [161]:
#they visit more than they get blood work done 
#there isn't bloodwork for every associated visit 
#average a1c levels vs num_visits 
#associate the bloodwork that we can for the visit 

df_summary["a1c_type"]=""
df_summary["a1c_level"]=""



for i in range(df_summary.shape[0]):
        mask=df_summary.loc[df_summary["patient_id"]==df_summary["patient_id"][i]]
        mask_2=df_a1c.loc[df_summary["patient_id"]==df_summary["patient_id"][i]]
        
        if mask == mask_2 and mask["visit_date"] - mask_2["obsdate"] <= np.timedelta64(2,'W'):
                df_summary["a1c_type"][i] = mask_2["date_of_a1c"]
                df_summary["a1c_level"][i]= mask_2["ac1_level"]

IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).